In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip3 install torch torchvision
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [0]:
!pip3 install --upgrade pip

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!cat /proc/meminfo

In [0]:
!cat /proc/cpuinfo

In [0]:
!ls drive/Collab/
import os
os.chdir("drive/Collab")

In [0]:
print('Setup complete')

In [0]:
os.chdir("deep_dream_pytorch")
!ls

In [0]:
#Begin deep_dream
!pip3 install --no-cache-dir -I pillow
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models
from torchvision import transforms, utils
import numpy as np
from PIL import Image, ImageFilter, ImageChops

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
net = models.vgg16
net = net(pretrained=True).cuda()
print(net)
modulelist = list(net.features.modules())

In [0]:
  def load_image(path):
    image = Image.open(path)
    plt.imshow(image)
    plt.title("Image loaded successfully")
    return image
  
  preprocess = transforms.Compose([transforms.Resize((224,224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
  def deprocess(image):
    return image * torch.Tensor([0.229, 0.224, 0.225]).cuda()  + torch.Tensor([0.485, 0.456, 0.406]).cuda()
  

In [0]:
def gradient_ascent(image, layer, iterations, lr):
  input = Variable(preprocess(image).unsqueeze(0).cuda(), requires_grad=True)
  net.zero_grad()
  for i in range(iterations):
    out = input
    for j in range(layer):
      out = modulelist[j+1](out)
      loss = out.norm()
      loss.backward(retain_graph=True)
      input.data = input.data + lr * input.grad.data
      
  input = input.data.squeeze()
  input.transpose_(0,1)
  input.transpose_(1,2)
  input = np.clip(deprocess(input),0,1)
  im = Image.fromarray(np.uint8(input*255))
  return im


In [0]:

def octaves(image, layer, iterations, lr, octave_scale, num_octaves):
  if num_octaves>0:
    image1 = image.filter(ImageFilter.GaussianBlur(2))
    if (image1.size[0]/octave_scale<1 or image1.size[1]/octave_scale<1):
      size = image1.size
    else:
      size = (int(image1.size[0]/octave_scale), int(image1.size[1]/octave_scale))
    
    image1 =  image1.resize(size, Image.ANTIALIAS)
    image1 = octaves(image1, layer, iterations, lr, octave_scale, num_octaves-1)
    size = (image.size[0], image.size[1])
    image1 = image1.resize(size, Image.ANTIALIAS)
    image = ImageChops.blend(image, image1, 0.6)
    print("---Octave Level : {} ---".format(num_octaves))
  img_result = gradient_ascent(image, layer, iterations, lr)
  img_result.resize(image.size)
  plt.imshow(img_result)
  return img_result

In [0]:
me = load_image("beagle.jpg")

In [0]:
me_5 = octaves(me, 12, 9, 0.01, 4, 1)